In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [ ]:
specimen1 = pd.read_csv('specimen1.csv')
specimen2 = pd.read_csv('specimen2.csv')
specimen3 = pd.read_csv('specimen3.csv')

data = pd.read_csv('FEA-FPU50-tensile.csv') # FEA data

In [ ]:
data1 = specimen1[specimen1['Displacement'] < 1]
plt.plot(data1['Displacement'], data1['Force'])

data2 = specimen2[specimen2['Displacement'] < 1]
plt.plot(data2['Displacement'], data2['Force'])

data3 = specimen3[specimen3['Displacement'] < 1]
plt.plot(data3['Displacement'], data3['Force'])

plt.title("Force Displacement Curve")
plt.xlabel("Displacement $[mm]$")
plt.ylabel("Force $[N]$")
plt.legend(['Specimen 1', 'Specimen 2', 'Specimen 3'])

In [ ]:
specimen_thickness = 3.4 # in mm
specimen_width = 6 # in mm
original_length = 65 # in mm

area = specimen_thickness * specimen_width


In [ ]:
def stress_strain(force, displacement):
    area = specimen_thickness * specimen_width # cross sectional area of the specimen

    stress = force / area
    strain = displacement / original_length # change in length / original

    return stress, strain

stress1, strain1 = stress_strain(data1['Force'], data1['Displacement'])
stress2, strain2 = stress_strain(data2['Force'], data2['Displacement'])
stress3, strain3 = stress_strain(data3['Force'], data3['Displacement'])

np.size(strain1)

In [ ]:
plt.plot(strain1, stress1)
plt.plot(strain2, stress2)
plt.plot(strain3, stress3)
plt.title("Stress Strain Curve")
plt.xlabel("Strain $[-]$")
plt.ylabel("Stress $[N/mm^2]$")
plt.legend(['Specimen 1', 'Specimen 2', 'Specimen 3'])

In [ ]:
model = LinearRegression()

stress2 = np.array(stress2).reshape(-1, 1)
strain2= np.array(strain2).reshape(-1, 1)
model.fit(strain2, stress2[0:65])

fit_line_data = model.predict(strain2)
plt.plot(strain2, fit_line_data, 'b--', label = 'Fit Line')
# plt.plot(strain2, stress2)
plt.plot(strain2, stress2[0:65], 'k-', label = 'Data', linewidth = 0.5)
plt.legend()

In [ ]:
# FEA data analysis

new_data = data[data['Displacement1'] < 1] # displacement less than 1 mm

new_force_data = new_data['Force1']
new_disp_data = new_data['Displacement1']

force_data = data['Force1']
disp_data = data['Displacement1']

new_data

In [ ]:
def stress_strain(force, displacement):
    area = specimen_thickness * specimen_width # cross sectional area of the specimen

    stress = force / area
    strain = displacement / original_length # change in length / original

    return stress, strain

stress, strain = stress_strain(new_force_data, new_disp_data)


stress_strain_data = {'Stress':stress, 'Strain': strain}
df = pd.DataFrame(stress_strain_data)
# df

area = specimen_thickness * specimen_width
Youngs_modulus_N = (df.iloc[-1]['Stress'] - df.iloc[0]['Stress']) / (df.iloc[-1]['Strain'] - df.iloc[0]['Strain'])

print("FEA: Young's Modulus (in MPa) = ", Youngs_modulus_N)
print("ASTM Type IV: Young's Modulus (in MPa) = ", model.coef_[0])


In [ ]:
plt.plot(disp_data, force_data, label = 'FEA data')
plt.plot(specimen2['Displacement'], specimen2['Force'], label = 'Specimen 2')
plt.title("Force Displacement Curve - FPU50")
plt.xlabel("Displacement $[mm]$")
plt.ylabel("Force $[N]$")
plt.legend()
plt.show()


In [ ]:
# df.to_csv('stress-strain-PA6-FEA.csv')